In [1]:
import pandas as pd

In [2]:
game_states_df = pd.read_csv('game_states.csv')

In [3]:
game_states_df.head()

,MatchId,MapName,RoundNum,RoundTick,BombPlantedA,BombPlantedB,AttackerSide,VictimSide,AttackerSteamId,EventType,...,ChangeClosestDistToA_T,ChangeClosestDistToB_T,ClosestDistToA_CT,ClosestDistToB_CT,ChangeClosestDistToA_CT,ChangeClosestDistToB_CT,RoundEndReason,CTBuyType,TBuyType,WinningSide
0,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,104,False,False,NaN,NaN,NaN,PositionUpdate,...,0.0,0.0,6.0,17.0,0.0,0.0,TargetBombed,Pistol,Pistol,T
1,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,362,False,False,NaN,NaN,NaN,PositionUpdate,...,1.0,-4.0,15.0,14.0,9.0,-3.0,TargetBombed,Pistol,Pistol,T
2,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,620,False,False,NaN,NaN,NaN,PositionUpdate,...,0.0,-7.0,7.0,13.0,-8.0,-1.0,TargetBombed,Pistol,Pistol,T
3,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,646,False,False,T,CT,7.656120e+16,DamageDealt,...,0.0,-7.0,7.0,13.0,-8.0,-1.0,TargetBombed,Pistol,Pistol,T
4,961Gamers-Nationals-2020-yalla-vs-we-sell-powe...,de_dust2,1,878,False,False,NaN,NaN,NaN,PositionUpdate,...,0.0,-2.0,9.0,11.0,2.0,-2.0,TargetBombed,Pistol,Pistol,T


In [4]:
# Split the dataset into train and test (on round so that events within a round aren't split across train and test)
from sklearn.model_selection import train_test_split

round_index = ['MatchId', 'MapName', 'RoundNum']
rounds_df = game_states_df[round_index].drop_duplicates()
train, test = train_test_split(rounds_df, test_size=0.2, random_state=10)
split_df = pd.merge(train, test, how='outer', indicator=True)
split_df['dataset'] = split_df._merge.map({'left_only': 'train', 'right_only': 'test'})
split_df.drop('_merge', axis=1, inplace=True)
split_df.index = pd.MultiIndex.from_arrays(split_df[round_index].values.T)
split_df.drop(round_index, axis=1, inplace=True)
split_df.head()

,,,dataset
BLAST-Premier-Spring-Showdown-2021-furia-vs-mibr-bo3,de_mirage,14,train
Perfect-World-League-Season1-2021-Qualifier2-checkmate-vs-shpl-bo3,de_overpass,13,train
ESEA-Premier-Season36-Australia-paradox-vs-order-bo3,de_vertigo,7,train
Funspark-ULTI-2021-Asia-Regional-Series-1-d13-vs-lynn-vision-bo3,de_nuke,8,train
European-Development-Championship-3-endpoint-vs-trident-bo3,de_train,14,train


In [5]:
model_parameters = ['MapName', 'RoundTick', 'TAlive', 'CTAlive', 'TTotalHp', 'CTTotalHp', 'BombPlantedA', 'BombPlantedB', 'ClosestDistToA_T', 'ClosestDistToB_T', 'ChangeClosestDistToA_T', 'ChangeClosestDistToB_T', 'ClosestDistToA_CT', 'ClosestDistToB_CT', 'ChangeClosestDistToA_CT', 'ChangeClosestDistToB_CT', 'CTBuyType', 'TBuyType', 'WinningSide']
data = game_states_df[model_parameters]
data.index = pd.MultiIndex.from_arrays(game_states_df[round_index].values.T)
data = data.merge(split_df, left_index=True, right_index=True)
data = pd.get_dummies(data, drop_first=True)
data

ng-2021-apeks-vs-oilers-bo3       de_nuke  28       7970   
                                                            28       8228   
                                                            28       8492   
                                                            28       8750   
                                                            28       8884   

                                                                TAlive  \
961Gamers-Nationals-2020-yalla-vs-we-sell-power... de_dust2 1      5.0   
                                                            1      5.0   
                                                            1      5.0   
                                                            1      5.0   
                                                            1      5.0   
...                                                                ...   
Telia-League-Spring-2021-apeks-vs-oilers-bo3       de_nuke  28     5.0   
                                                            28     5.0   
                                                            28     5.0   
                                                            28     5.0   
                                                            28     5.0   

                                                                CTAlive  \
961Gamers-Nationals-2020-yalla-vs-we-sell-power... de_dust2 1       5.0   
                                                            1       5.0   
                                                            1       5.0   
                                                            1       5.0   
                                                            1       5.0   
...                                                                 ...   
Telia-League-Spring-2021-apeks-vs-oilers-bo3       de_nuke  28      5.0   
                                                            28      5.0   
                                                            28      5.0   
                                                            28      5.0   
                                                            28      5.0   

                                                                TTotalHp  \
961Gamers-Nationals-2020-yalla-vs-we-sell-power... de_dust2 1        5.0   
                                                            1        5.0   
                                                            1        5.0   
                                                            1      500.0   
                                                            1      500.0   
...                                                                  ...   
Telia-League-Spring-2021-apeks-vs-oilers-bo3       de_nuke  28     100.0   
                                                            28     100.0   
                                                            28     100.0   
                                                            28     100.0   
                                                            28     100.0   

                                                                CTTotalHp  \
961Gamers-Nationals-2020-yalla-vs-we-sell-power... de_dust2 1         5.0   
                                                            1         5.0   
                                                            1         5.0   
                                                            1       415.0   
                                                            1       415.0   
...                                                                   ...   
Telia-League-Spring-2021-apeks-vs-oilers-bo3       de_nuke  28      100.0   
                                                            28      100.0   
                                                            28      100.0   
                                                            28      100.0   
                                                            28        0.0   

   

In [6]:
data.isna().sum()

RoundTick                    0
TAlive                       0
CTAlive                      0
TTotalHp                     0
CTTotalHp                    0
BombPlantedA                 0
BombPlantedB                 0
ClosestDistToA_T           292
ClosestDistToB_T           292
ChangeClosestDistToA_T       0
ChangeClosestDistToB_T       0
ClosestDistToA_CT          132
ClosestDistToB_CT          132
ChangeClosestDistToA_CT      0
ChangeClosestDistToB_CT      0
MapName_de_inferno           0
MapName_de_mirage            0
MapName_de_nuke              0
MapName_de_overpass          0
MapName_de_train             0
MapName_de_vertigo           0
CTBuyType_Full Buy           0
CTBuyType_Full Eco           0
CTBuyType_Half Buy           0
CTBuyType_Pistol             0
TBuyType_Full Buy            0
TBuyType_Full Eco            0
TBuyType_Half Buy            0
TBuyType_Pistol              0
WinningSide_T                0
dataset_train                0
dtype: int64

In [7]:
rounds_with_na = data[data.isna().any(axis=1)].index.unique()
data.drop(rounds_with_na, inplace=True)

In [8]:
# Min/Max Scaling for health and alive can be done across train and test
data[['TAlive', 'CTAlive']] = data[['TAlive', 'CTAlive']] / 5
data[['TTotalHp', 'CTTotalHp']] = data[['TTotalHp', 'CTTotalHp']] / 500

In [9]:
from sklearn.preprocessing import StandardScaler

features = ['RoundTick', 'ClosestDistToA_T', 'ClosestDistToB_T', 'ChangeClosestDistToA_T', 'ChangeClosestDistToB_T', 'ClosestDistToA_CT', 'ClosestDistToB_CT', 'ChangeClosestDistToA_CT', 'ChangeClosestDistToB_CT']
scaler = StandardScaler().fit(data[data.dataset_train == 1][features])
data[features] = scaler.transform(data[features])

data.describe()

,RoundTick,TAlive,CTAlive,TTotalHp,CTTotalHp,ClosestDistToA_T,ClosestDistToB_T,ChangeClosestDistToA_T,ChangeClosestDistToB_T,ClosestDistToA_CT,...,CTBuyType_Full Buy,CTBuyType_Full Eco,CTBuyType_Half Buy,CTBuyType_Pistol,TBuyType_Full Buy,TBuyType_Full Eco,TBuyType_Half Buy,TBuyType_Pistol,WinningSide_T,dataset_train
count,2.711351e+06,2711351.0,2711351.0,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,...,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06,2.711351e+06
mean,-4.910066e-04,1.0,1.0,5.133001e-01,5.453355e-01,-2.653697e-03,-1.622921e-03,-5.578527e-05,-3.650797e-05,-3.491638e-03,...,6.742550e-01,1.780662e-02,2.054083e-01,6.133179e-02,7.677427e-01,1.478672e-02,1.229210e-01,6.133179e-02,5.195233e-01,7.996928e-01
std,9.998573e-01,0.0,0.0,3.429283e-01,3.460480e-01,9.962885e-01,9.980832e-01,1.001131e+00,1.002344e+00,9.962143e-01,...,4.686526e-01,1.322481e-01,4.039997e-01,2.399379e-01,4.222723e-01,1.206983e-01,3.283466e-01,2.399379e-01,4.996188e-01,4.002303e-01
min,-1.567344e+00,1.0,1.0,0.000000e+00,0.000000e+00,-1.383205e+00,-1.712076e+00,-1.630297e+01,-1.392827e+01,-1.195596e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-8.347788e-01,1.0,1.0,2.000000e-01,2.020000e-01,-8.422006e-01,-8.307668e-01,-1.968061e-01,-1.493300e-01,-5.702895e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
50%,-1.404021e-01,1.0,1.0,5.640000e-01,6.000000e-01,-2.239095e-01,5.054229e-02,1.960271e-01,1.208452e-01,-3.201669e-01,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
75%,7.169609e-01,1.0,1.0,8.000000e-01,8.320000e-01,5.489543e-01,6.674587e-01,1.960271e-01,1.208452e-01,1.800785e-01,...,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
max,3.398373e+00,1.0,1.0,1.000000e+00,1.000000e+00,3.949555e+00,4.192695e+00,1.590936e+01,1.471031e+01,7.058452e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [10]:
train = data[data.dataset_train == 1].drop('dataset_train', axis=1)
test = data[data.dataset_train == 0].drop('dataset_train', axis=1)
X_train = train.drop('WinningSide_T', axis=1)
y_train = train.WinningSide_T
X_test = test.drop('WinningSide_T', axis=1)
y_test = test.WinningSide_T
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2168248, 29)
(2168248,)
(543103, 29)
(543103,)


## Fit Models

In [11]:
def hyperparam_search(model, model_kwargs, search_kwargs):
    m = model(**model_kwargs, )
    

### Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
import pickle

lr_model = LogisticRegression(max_iter=1000, solver='saga', penalty='none', random_state=10).fit(X_train, y_train)

with open('logistic.pickle', 'wb') as f:
    pickle.dump(lr_model, f)

### CatBoost

In [13]:
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss

cb_params = {
    "learning_rate": [0.05, 0.1, 0.5, 1],
    "depth": [6, 8, 10],
    "l2_leaf_reg": [1, 3, 5, 7, 9]
}


best_score = None

for lr_param in cb_params['learning_rate']:
    for depth_param in cb_params['depth']:
        for leaf_param in cb_params['l2_leaf_reg']:
            print(f'Trying hyperparameters: learning_rate ({lr_param}), depth ({depth_param}), l2_leaf_reg ({leaf_param})')

            this_model = CatBoostClassifier(
                iterations=100,
                custom_metric=['Logloss'],
                learning_rate=lr_param,
                depth=depth_param,
                l2_leaf_reg=leaf_param,
                verbose=False,
                task_type='GPU',
                devices='0:1'
            ).fit(X_train, y_train)

            this_score = -1 * log_loss(y_test, this_model.predict(X_test))
            print('Log Loss:', this_score)
            
            if best_score is None or this_score > best_score:
                best_score = this_score
                best_model = this_model

cb_model = best_model
cb_model.save_model('catboost.model')

Trying hyperparameters: learning_rate (0.05), depth (6), l2_leaf_reg (1)
Log Loss: -8.295701721647669
Trying hyperparameters: learning_rate (0.05), depth (6), l2_leaf_reg (3)
Log Loss: -8.26664295014776
Trying hyperparameters: learning_rate (0.05), depth (6), l2_leaf_reg (5)
Log Loss: -8.300407480836128
Trying hyperparameters: learning_rate (0.05), depth (6), l2_leaf_reg (7)
Log Loss: -8.273953738735683
Trying hyperparameters: learning_rate (0.05), depth (6), l2_leaf_reg (9)
Log Loss: -8.269692907857294
Trying hyperparameters: learning_rate (0.05), depth (8), l2_leaf_reg (1)
Log Loss: -8.238848302866426
Trying hyperparameters: learning_rate (0.05), depth (8), l2_leaf_reg (3)
Log Loss: -8.237193945690105
Trying hyperparameters: learning_rate (0.05), depth (8), l2_leaf_reg (5)
Log Loss: -8.229054753580513
Trying hyperparameters: learning_rate (0.05), depth (8), l2_leaf_reg (7)
Log Loss: -8.226002463786038
Trying hyperparameters: learning_rate (0.05), depth (8), l2_leaf_reg (9)
Log Loss: 

In [14]:
best_model.score(X_test, y_test)

0.7628092645409803

### XGBoost

In [15]:
from xgboost import XGBClassifier

xgb_params = { 
 'max_depth': [6, 8, 10, 12, 14],
 'colsample_bytree': [0.2, 0.4, 0.6, 0.8],
 'learning_rate': [0.01, 0.05, 0.1, 0.2],
 'min_child_weight': [1, 3, 5, 7]
}

best_score = None

for depth_param in xgb_params['max_depth']:
    for cs_param in xgb_params['colsample_bytree']:
        for lr_param in xgb_params['learning_rate']:
            for mcw_param in xgb_params['min_child_weight']:
                print(f'Trying hyperparameters: max_depth ({depth_param}), colsample_bytree ({cs_param}), learning_rate ({lr_param}), min_child_weight ({mcw_param})')
                this_model = XGBClassifier(
                    n_estimators=100,
                    objective='binary:logistic',
                    tree_method='gpu_hist',
                    gpu_id=0,
                    max_depth=depth_param,
                    colsample_bytree=cs_param,
                    learning_rate=lr_param,
                    min_child_weight=mcw_param,
                    verbosity=0
                ).fit(X_train, y_train)

                this_score = -1 * log_loss(y_test, this_model.predict(X_test))
                print('Log Loss:', this_score)

                if best_score is None or this_score > best_score:
                    best_score = this_score
                    best_model = this_model


xgb_model = best_model
xgb_model.save_model('xgboost.model')

s deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
Log Loss: -8.657322800414844
Trying hyperparameters: max_depth (14), colsample_bytree (0.4), learning_rate (0.2), min_child_weight (7)
/home/ctshinn/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
Log Loss: -8.7012674782978
Trying hyperparameters: max_depth (14), colsampl

### Evaluate For Different Loss/Scoring Metrics

In [16]:
from sklearn.metrics import brier_score_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

def score(y_true, y_pred):
    return log_loss(y_true, y_pred), brier_score_loss(y_true, y_pred[:, 1]), roc_auc_score(y_true, y_pred[:, 1]), accuracy_score(y_true, y_pred[:, 1] >= 0.5)

In [17]:
scores = []

for name, model in zip(['LinearRegression', 'CatBoost', 'XGBoost'], [lr_model, cb_model, xgb_model]):
    y_test_pred = model.predict_proba(X_test)
    scores.append([name] + list(score(y_test, y_test_pred)))

In [18]:


scores_df = pd.DataFrame(scores, columns=['Model', 'LogLoss', 'BrierScore', 'AUC', 'Accuracy'])
scores_df

,Model,LogLoss,BrierScore,AUC,Accuracy
0,LinearRegression,0.470612,0.157843,0.849490,0.751981
1,CatBoost,0.452203,0.151300,0.861704,0.762809
2,XGBoost,0.451108,0.151030,0.861950,0.762837
